<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Vectorizing-tabular-fields" data-toc-modified-id="Vectorizing-tabular-fields-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Vectorizing tabular fields</a></span></li></ul></div>

# Vectorizing tabular fields

This notebook covers simple methods to vectorize tabular data using the same dataset as in the other examples, but this time ignoring the text of the question.

In [1]:
import pandas as pd
from pathlib import Path
import sys
sys.path.append('..')

# import warnings
# warnings.filterwarnings('ignore')

from ml_editor.ch4_data_processing import get_normalized_series
data_path = Path("../raw_data/writers.csv")
df = pd.read_csv(data_path)

Let's pretend we wanted to predict the score from the tags, number of comments, and question creation date. Here is what the data looks like

In [4]:
df['is_question'] = df['PostTypeId'] == 1

In [5]:
tabular_df = df[df['is_question']][['Tags', 'CommentCount', 'CreationDate', 'Score']]
tabular_df.head()

,Tags,CommentCount,CreationDate,Score
0,<resources><first-time-author>,7,2010-11-18T20:40:32.857,32
1,<fiction><grammatical-person><third-person>,0,2010-11-18T20:42:31.513,23
2,<publishing><novel><agent>,1,2010-11-18T20:43:28.903,36
3,<plot><short-story><planning><brainstorming>,1,2010-11-18T20:43:59.693,34
4,<fiction><genre><categories>,1,2010-11-18T20:45:44.067,22


In order to use this data as input to a model, we need to give it a suitable numerical representation. To do so, we will do three things here:

1. Normalize numerical input features to limit the impact of outliers

2. Transform the date feature in a way that makes it easier to understand for a model.

3. Get dummy variables from categorical features so a model can ingest them.

First, we normalize the data to reduce the effect of outliers on downstream model performance.

In [7]:
# get_normalized_series is the method from ch4_data_processing. Code - 
# return (df[col] - df[col].mean() / df[col].std())


tabular_df['NormComment'] = get_normalized_series(tabular_df, 'CommentCount')
tabular_df['NormScore'] = get_normalized_series(tabular_df, 'Score')

In [8]:
tabular_df.head()

,Tags,CommentCount,CreationDate,Score,NormComment,NormScore
0,<resources><first-time-author>,7,2010-11-18T20:40:32.857,32,6.113912,31.338937
1,<fiction><grammatical-person><third-person>,0,2010-11-18T20:42:31.513,23,-0.886088,22.338937
2,<publishing><novel><agent>,1,2010-11-18T20:43:28.903,36,0.113912,35.338937
3,<plot><short-story><planning><brainstorming>,1,2010-11-18T20:43:59.693,34,0.113912,33.338937
4,<fiction><genre><categories>,1,2010-11-18T20:45:44.067,22,0.113912,21.338937


Now, let's represent dates in a way that would make it easier for a model to extract patterns (see chapter 4 of the attached book for more information on why we chose these particular features.)

In [9]:
# Convert our date to a pandas datetime
tabular_df['date'] = pd.to_datetime(tabular_df['CreationDate'])

# Extract meaningful features from the datetime object
tabular_df['year'] = tabular_df['date'].dt.year
tabular_df['month'] = tabular_df['date'].dt.month
tabular_df['day'] = tabular_df['date'].dt.day
tabular_df['hour'] = tabular_df['date'].dt.hour

In [10]:
tabular_df.head()

,Tags,CommentCount,CreationDate,Score,NormComment,NormScore,date,year,month,day,hour
0,<resources><first-time-author>,7,2010-11-18T20:40:32.857,32,6.113912,31.338937,2010-11-18 20:40:32.857,2010,11,18,20
1,<fiction><grammatical-person><third-person>,0,2010-11-18T20:42:31.513,23,-0.886088,22.338937,2010-11-18 20:42:31.513,2010,11,18,20
2,<publishing><novel><agent>,1,2010-11-18T20:43:28.903,36,0.113912,35.338937,2010-11-18 20:43:28.903,2010,11,18,20
3,<plot><short-story><planning><brainstorming>,1,2010-11-18T20:43:59.693,34,0.113912,33.338937,2010-11-18 20:43:59.693,2010,11,18,20
4,<fiction><genre><categories>,1,2010-11-18T20:45:44.067,22,0.113912,21.338937,2010-11-18 20:45:44.067,2010,11,18,20


And finally lets transform tags into dummy variables using pandas get_dummies function, with each tag being assigned an 
index that will take the value "1" only if it is present in the given row.

In [12]:
# Select our tags, represented as strings, and transform them into arrays of tags

tags = tabular_df['Tags']
clean_tags = tags.str.split('><').apply(
    lambda x: [a.strip('<').strip('>') for a in x])

# Use pandas' get_dummies to get dummy values 
# select only tags that appear over 500 times

tag_columns = pd.get_dummies(clean_tags.apply(pd.Series).stack()).sum(level=0)

In [13]:
print(type(tag_columns))

<class 'pandas.core.frame.DataFrame'>


In [14]:
tag_columns.head(2)

,3-acts,academic-writing,accessibility,acronyms,action,administration-guides,agent,altered-perspective,alternative-history,amazon,...,wikipedia,word-choice,wordcount,world-building,writer,writers-block,writing-groups,writing-instruments,young-adult,young-author
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
all_tags = tag_columns.astype(bool).sum(axis=0).sort_values(ascending=False)

In [19]:
top_tags = all_tags[all_tags > 500]
top_tags

creative-writing         1838
fiction                  1514
style                    1170
characters                888
technique                 703
novel                     686
publishing                655
character-development     581
plot                      567
dtype: int64

In [20]:
top_tag_columns = tag_columns[top_tags.index]
top_tag_columns.head()

,creative-writing,fiction,style,characters,technique,novel,publishing,character-development,plot
0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0
2,0,0,0,0,0,1,1,0,0
3,0,0,0,0,0,0,0,0,1
4,0,1,0,0,0,0,0,0,0


In [21]:
# Add our tags back into our initial DataFrame

final = pd.concat([tabular_df, top_tag_columns], axis=1)
final.head()


,Tags,CommentCount,CreationDate,Score,NormComment,NormScore,date,year,month,day,hour,creative-writing,fiction,style,characters,technique,novel,publishing,character-development,plot
0,<resources><first-time-author>,7,2010-11-18T20:40:32.857,32,6.113912,31.338937,2010-11-18 20:40:32.857,2010,11,18,20,0,0,0,0,0,0,0,0,0
1,<fiction><grammatical-person><third-person>,0,2010-11-18T20:42:31.513,23,-0.886088,22.338937,2010-11-18 20:42:31.513,2010,11,18,20,0,1,0,0,0,0,0,0,0
2,<publishing><novel><agent>,1,2010-11-18T20:43:28.903,36,0.113912,35.338937,2010-11-18 20:43:28.903,2010,11,18,20,0,0,0,0,0,1,1,0,0
3,<plot><short-story><planning><brainstorming>,1,2010-11-18T20:43:59.693,34,0.113912,33.338937,2010-11-18 20:43:59.693,2010,11,18,20,0,0,0,0,0,0,0,0,1
4,<fiction><genre><categories>,1,2010-11-18T20:45:44.067,22,0.113912,21.338937,2010-11-18 20:45:44.067,2010,11,18,20,0,1,0,0,0,0,0,0,0


In [23]:
col_to_keep = ['year', 'month', 'day', 'hour', 'NormComment',
               'NormScore'] + list(top_tags.index)
final_features = final[col_to_keep]

In [24]:
final_features.head()

,year,month,day,hour,NormComment,NormScore,creative-writing,fiction,style,characters,technique,novel,publishing,character-development,plot
0,2010,11,18,20,6.113912,31.338937,0,0,0,0,0,0,0,0,0
1,2010,11,18,20,-0.886088,22.338937,0,1,0,0,0,0,0,0,0
2,2010,11,18,20,0.113912,35.338937,0,0,0,0,0,1,1,0,0
3,2010,11,18,20,0.113912,33.338937,0,0,0,0,0,0,0,0,1
4,2010,11,18,20,0.113912,21.338937,0,1,0,0,0,0,0,0,0


Voila! Our tabular data is now ready to be used for a model.